<a id='top'></a>
<h1 style="font-family:Bahnschrift Condensed;text-align:center; font-size:240%;">Mauritanian Number-plate recognition challenge</h1>
<h2 style="font-family:Bahnschrift Condensed;text-align:center; font-size:240%;">2024 National Data Science Competition by RIMAI: License Plate Recognition in Mauritania - Data Science Phase</h2>
<div style="text-align:center;">
    <img src="https://www.rim-ai.com/assets/logo.png" style="width:30%; height:auto;" alt="Logo RIM AI">
</div>

- **El Moustapha Cheikh Jiddou** 
- **Pseudo: Jiddou26**
- **Email:** elmoustapha.cheikh.jiddou@gmail.com

<p style="font-family:Bahnschrift Condensed;font-size:120%; text-align:justify;">
    The second phase of the 2024 National Data Science Competition by RIMAI shifts the focus to the heart of the challenge: data science. After gathering diverse and high-quality images in the first phase, this stage is dedicated to developing and fine-tuning algorithms for the automatic recognition of Mauritanian license plates.
</p>

<p style="font-family:Bahnschrift Condensed;font-size:120%; text-align:justify;">
    Participants are invited to leverage their data science skills to create robust models capable of accurately identifying and reading license plates under various conditions. This phase is crucial, as it will test the effectiveness of the algorithms in real-world scenarios, pushing the boundaries of AI and machine learning in the context of Mauritanian license plate recognition.
</p>

<p style="font-family:Bahnschrift Condensed;font-size:120%; text-align:justify;">
    The competition aims to drive innovation in computer vision, with a particular emphasis on enhancing the accuracy, efficiency, and scalability of the solutions. The challenge is designed not only to assess technical prowess but also to encourage creativity in overcoming the unique challenges presented by this application.
</p>

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 1: Importing Necessary Libraries</b></span></div>

In [4]:
import os
import cv2
import numpy as np
from shutil import copyfile
from easyocr import Reader
import pytesseract
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import math
import pandas as pd

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 2: License Plate Detection and Transformation Functions</b></span></div>

In [5]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 3: Processing and Annotating Images</b></span></div>

In [6]:
def process_single_image(image_path, output_dir):
    car = cv2.imread(image_path)
    gray = cv2.cvtColor(car, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 200)
    cont, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cont = sorted(cont, key=cv2.contourArea, reverse=True)[:5]

    plate_cnt = None
    for c in cont:
        arc = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * arc, True)
        if len(approx) == 4:
            plate_cnt = approx
            break

    if plate_cnt is None:
        print(f"No license plate detected in {image_path}.")
        return None

    warped_plate = four_point_transform(gray, plate_cnt.reshape(4, 2))
    plate = cv2.resize(warped_plate, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    reader = Reader(['en'])
    detection = reader.readtext(plate)

    if len(detection) == 0:
        print(f"Unable to read license plate text in {image_path}.")
        return None

    text = detection[0][1]
    filtered_text = ''.join([c for c in text if c.isalnum()])
    output_filename = os.path.join(output_dir, f'{filtered_text}.jpg')
    cv2.imwrite(output_filename, car)

    return filtered_text, output_filename

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 4: Processing Images in Folders</b></span></div>

In [ ]:
train_data_dir = "train_data"
test_data_dir = "test_data"
output_dir = "processed_images"
submission_dir = "submission"

# Create necessary directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(submission_dir, exist_ok=True)

results = []

# Process training data
for filename in os.listdir(train_data_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(train_data_dir, filename)
        result = process_single_image(image_path, output_dir)

        if result:
            plate_number, output_filename = result
            print(f"Processed: {filename} -> Saved as: {output_filename}")
            results.append((filename, plate_number))
        else:
            print(f"Processed: {filename} -> No license plate detected or unable to read text.")

# Save the results to CSV
results_df = pd.DataFrame(results, columns=['img_id', 'plate_number'])
results_df.to_csv(os.path.join(submission_dir, 'predictions.csv'), index=False)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


No license plate detected in train_data\img_1.jpg.
Processed: img_1.jpg -> No license plate detected or unable to read text.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processed: img_10.jpg -> Saved as: processed_images\5115AMOdi.jpg
Processed: img_100.jpg -> Saved as: processed_images\7812AA02.jpg
No license plate detected in train_data\img_102.jpg.
Processed: img_102.jpg -> No license plate detected or unable to read text.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Unable to read license plate text in train_data\img_277.jpg.
Processed: img_277.jpg -> No license plate detected or unable to read text.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processed: img_278.jpg -> Saved as: processed_images\1693AB06.jpg
Unable to read license plate text in train_data\img_279.jpg.
Processed: img_279.jpg -> No license plate detected or unable to read text.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 5: Loading Data and Labels</b></span></div>

In [ ]:
def load_data_and_labels(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            label = filename.split('_')[0]
            labels.append(label)
    return np.array(images), np.array(labels)

X_train, y_train = load_data_and_labels(output_dir)
X_test, y_test = load_data_and_labels(test_data_dir)

X_train = X_train / 255.0
X_test = X_test / 255.0

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 6: Building and Training the CNN Model</b></span></div>

In [ ]:
# Improved model
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# Augment the training data
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Train the model with data augmentation
history = model.fit(
    train_datagen.flow(X_train_split, y_train_split, batch_size=32),
    validation_data=val_datagen.flow(X_val_split, y_val_split),
    epochs=10
)

# Display the model architecture
model.summary()

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 7: Evaluating the Model</b></span></div>

In [ ]:
# Encoding training labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Diagnose potential label mismatches
try:
    y_test_transformed = label_encoder.transform(y_test)
except ValueError as e:
    print(f"Error transforming y_test: {e}")
    valid_test_indices = [i for i, label in enumerate(y_test) if label in label_encoder.classes_]
    y_test_transformed = label_encoder.transform([y_test[i] for i in valid_test_indices])
    X_test = X_test[valid_test_indices]

# Filter the test data to include only labels present in training data
train_labels_set = set(y_train_encoded)
filtered_indices = [i for i, label in enumerate(y_test_transformed) if label in train_labels_set]
if len(filtered_indices) == 0:
    print("No matching labels between training and test sets. Skipping evaluation.")
else:
    X_test_filtered = X_test[filtered_indices]
    y_test_filtered = y_test_transformed[filtered_indices]

    # Convert labels to one-hot encoding
    num_classes = len(np.unique(y_train_encoded))
    y_train_one_hot = to_categorical(y_train_encoded, num_classes)
    y_test_one_hot = to_categorical(y_test_filtered, num_classes)

    # Evaluate the model on filtered test data
    loss, accuracy = model.evaluate(X_test_filtered, y_test_one_hot)
    print(f'Loss: {loss}, Accuracy: {accuracy}')


<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 8: Testing the Model on a Single Image</b></span></div>

In [ ]:
image_path = 'DataChallenge.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (128, 128))

# Normalize the image
image = image / 255.0

# Make a prediction
prediction = model.predict(np.expand_dims(image, axis=0))

# Convert probabilities to labels
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])

print(f'Predicted label: {predicted_label}')

<div align="center"><span style="font-family:Bahnschrift Condensed;font-size:40px"><b>Section 9: Advanced License Plate Detection and Visualization</b></span></div>

In [ ]:
def process_single_image_advanced(image_path, output_path):
    reader = Reader(['en'], gpu=False, verbose=False)
    
    car = cv2.imread(image_path)
    car = cv2.resize(car, (800, 600))
    gray = cv2.cvtColor(car, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 200)
    cont, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cont = sorted(cont, key=cv2.contourArea, reverse=True)[:5]

    plate_cnt = None
    for c in cont:
        arc = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * arc, True)
        if len(approx) == 4:
            plate_cnt = approx
            break

    if plate_cnt is None:
        print("No license plate detected.")
        return

    warped_plate = four_point_transform(gray, plate_cnt.reshape(4, 2))
    plate = cv2.resize(warped_plate, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    plate = cv2.equalizeHist(plate)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    plate = cv2.morphologyEx(plate, cv2.MORPH_CLOSE, kernel)
    detection = reader.readtext(plate)

    if len(detection) == 0:
        text = "Impossible de lire le texte de la plaque d'immatriculation"
        cv2.putText(car, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)
    else:
        text = detection[0][1]
        filtered_text = ''.join([c for c in text if c.isalnum()])
        if len(filtered_text) > 8:
            filtered_text = filtered_text[:8]
        cv2.drawContours(car, [plate_cnt], -1, (0, 255, 0), 3)
        display_text = f"{filtered_text} ({detection[0][2] * 100:.2f}%)"
        cv2.putText(car, display_text, (plate_cnt[0][0][0], plate_cnt[0][0][1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        print(display_text)

    cv2.imwrite(output_path, car)
    cv2.imwrite('warped_plate.jpg', plate)

# Test with a single image
process_single_image_advanced('hamada.jpg', 'annotated_test.jpg')

# Display images with matplotlib
annotated_image = cv2.imread('annotated_test.jpg')
warped_plate = cv2.imread('warped_plate.jpg', cv2.IMREAD_GRAYSCALE)

plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(warped_plate, cmap='gray')
plt.title('License Plate')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.title('Annotated Image')
plt.axis('off')

plt.show()